### **Data Transformation (or data augmentation) as per the AlexNet paper** ##

data augmentation (only image translation and horizontal reflection; read section 4.1 
on the paper)
- Random crop of 224x224 from the 256x256 image
- Random horizontal flipping

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import random